In [1]:
from dataloader import DataLoader
import pandas as pd
import numpy as np

In [2]:
loader = DataLoader()

In [3]:
loader.load_sells_data_from_file('sprzedaz.csv')
df_filtered = loader = loader.prepare_sells_data()


In [7]:
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# 2. Kolumny miesięczne
miesiace_kolumny = [str(i) for i in range(96, 0, -1)]

# 3. Agregacja do poziomu produktu
df_agg = df_filtered.groupby("KodProduktu")[miesiace_kolumny].sum().reset_index()

# 4. Przekształcenie na długi format
df_long = df_agg.melt(id_vars="KodProduktu", var_name="miesiac_ago", value_name="sprzedaz")
df_long["miesiac_ago"] = df_long["miesiac_ago"].astype(int)
df_long["czas"] = 97 - df_long["miesiac_ago"]

# 5. Tworzenie lagów
lags = [1, 2, 3]
for lag in lags:
    df_long[f"lag_{lag}"] = df_long.groupby("KodProduktu")["sprzedaz"].shift(lag)

df_lags = df_long.dropna().copy()

#Dodane cechy

# 1. Rolling średnie
df_long["rolling_mean_3"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.rolling(window=3).mean())
df_long["rolling_mean_6"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.rolling(window=6).mean())

# 2. Trend: różnica między t a t-3
df_long["trend_3"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.diff(3))

# 3. Odchylenie standardowe (zmienność)
df_long["std_3"] = df_long.groupby("KodProduktu")["sprzedaz"].transform(lambda x: x.rolling(3).std())

# 4. Usuń brakujące dane (początki serii)
df_lags = df_long.dropna().copy()
#Dodane cechy

# 6. Kodowanie KodProduktu
le = LabelEncoder()
df_lags["KodProduktu_enc"] = le.fit_transform(df_lags["KodProduktu"])

# 7. Dane treningowe
cechy_dodatkowe = ["rolling_mean_3", "rolling_mean_6", "trend_3", "std_3"]

X = df_lags[[f"lag_{l}" for l in lags] + cechy_dodatkowe + ["KodProduktu_enc"]]
y = df_lags["sprzedaz"]

# 8. Trening modelu XGBoost
model = xgb.XGBRegressor(n_estimators=100, max_depth=5, learning_rate=0.1, random_state=42)
model.fit(X, y)

# 9. Prognozowanie dla wybranego produktu
kod = "2009"
kod_enc = le.transform([kod])[0]

# Dane historyczne tylko dla tego produktu, posortowane od najnowszych
df_kod = df_lags[df_lags["KodProduktu"] == kod].sort_values("czas", ascending=False)
last_values = df_kod.head(6)["sprzedaz"].tolist()  # potrzebne do rolling 6

preds = []

for i in range(3):
    # LAGI: ostatnie 3 wartości
    lag_1 = last_values[-1]
    lag_2 = last_values[-2]
    lag_3 = last_values[-3]

    # CECHY DODATKOWE:
    rolling_mean_3 = pd.Series(last_values[-3:]).mean()
    rolling_mean_6 = pd.Series(last_values[-6:]).mean()
    trend_3 = last_values[-1] - last_values[-4]  # różnica t - t-3
    std_3 = pd.Series(last_values[-3:]).std()

    # finalny wektor cech:
    x_input = [lag_1, lag_2, lag_3,
               rolling_mean_3, rolling_mean_6, trend_3, std_3,
               kod_enc]

    y_pred = model.predict([x_input])[0]
    preds.append(y_pred)

    # dodajemy nową predykcję do sekwencji
    last_values.append(y_pred)

# 10. Wynik
for i, p in enumerate(preds):
    print(f"Prognozowana sprzedaż produktu {kod} za miesiąc {i} od teraz: {p:.2f}")

Prognozowana sprzedaż produktu 2009 za miesiąc 0 od teraz: 311.82
Prognozowana sprzedaż produktu 2009 za miesiąc 1 od teraz: 274.25
Prognozowana sprzedaż produktu 2009 za miesiąc 2 od teraz: 244.06
